# Nuclear chromatin phenotypes of PBMCs reflects the treatment effect of proton therapy (Glioma)

---
This notebook summarizes the analysis corresponding to the results presented in figure 4 of the paper for Glioma patients. It can be used to rerun the analysis and regenerate the corresponding panels.

---

## 0. Environmental setup

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import matplotlib as mpl

mpl.rcParams["figure.dpi"] = 1200

import sys

sys.path.append("../..")
from src.utils.notebooks.eda import *
from src.utils.notebooks.figure3 import *
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score, StratifiedGroupKFold

seed = 1234
random.seed(1234)
np.random.seed(1234)

%reload_ext nb_black

In [ ]:
nuc_feature_desc = pd.read_csv(
    "../../data/chrometric_feature_description.csv", index_col=0
)
feature_name_dict = dict(
    zip(
        list(nuc_feature_desc.loc[:, "feature"]),
        list(nuc_feature_desc.loc[:, "long_name"]),
    )
)
feature_color_dict = {
    "morphology": "b",
    "intensity": "g",
    "boundary": "r",
    "texture": "c",
    "chromatin condensation": "m",
    "moments": "y",
    np.nan: "k",
}
feature_color_dict = {
    feature: feature_color_dict[category]
    for (feature, category) in zip(
        list(nuc_feature_desc.loc[:, "long_name"]),
        list(nuc_feature_desc.loc[:, "category"]),
    )
}

In [ ]:
color_palette = {
    "prior": "orange",
    "during": "gold",
    "end": "saddlebrown",
}

---

## 1. Read in data

In this notebook we assess the differences of the cell states of PBMCs at three different time points of the proton therapy treatment: before, during (~3 weeks in) and at the end of the treatment (final week of treatment). To this end, we obtained PBMCs of 8 Glioma patients stained them for DNA, gH2AX and Lamin A/C and obtained fluorescent images.

First, we read in the required data set that describe each PBMCs by a number of hand-crafted features extracted from the fluorescent images of the cells.

In [ ]:
all_data = pd.read_csv("../../data/treated_population_data.csv", index_col=0)
all_data = all_data.loc[all_data.cancer == "Glioma"]
all_data = preprocess_data(all_data, remove_constant_features=False)
all_data = all_data.rename(columns=feature_name_dict)
len(all_data)

In [ ]:
fig, ax = plt.subplots(figsize=[12, 4], ncols=2)
tp_order = ["prior", "during", "end"]
sample_order = np.unique(all_data.loc[:, "sample"])
ax = ax.flatten()
ax[0] = sns.countplot(
    x="sample",
    data=all_data,
    ax=ax[0],
    order=sample_order,
    hue_order=tp_order,
    hue="timepoint",
    palette=color_palette,
)
ax[0].legend([], [], frameon=False)
ax[0].set_xlabel("ID of the biological sample")
ax[0].set_title("Distribution of biological samples")
for tick in ax[0].get_xticklabels():
    tick.set_rotation(45)

ax[1] = sns.countplot(
    x="timepoint",
    hue="timepoint",
    data=all_data,
    ax=ax[1],
    order=tp_order,
    dodge=False,
    palette=color_palette,
)
ax[1].set_xlabel("Timepoint with respect to the treatment")
ax[1].set_title("Distribution of the different timepoints")
ax[1].legend(loc="lower right")

plt.show()
plt.close()

___

#### Subsampling

We next subsample the data set such that for each timepoint we have the same number of nuclei in the data set. Additionally, we ensure that the individual timepoint population are approximately uniformly represented by the different biological (patient) samples.

In [ ]:
sampled_data = get_stratified_data(
    all_data,
    id_column="id",
    cond_column="timepoint",
    seed=1234,
)

In [ ]:
fig, ax = plt.subplots(figsize=[12, 4], ncols=2)
tp_order = ["prior", "during", "end"]
sample_order = np.unique(all_data.loc[:, "sample"])
ax = ax.flatten()
ax[0] = sns.countplot(
    x="sample",
    data=sampled_data,
    ax=ax[0],
    order=sample_order,
    hue_order=tp_order,
    hue="timepoint",
    palette=color_palette,
)
ax[0].legend([], [], frameon=False)
ax[0].set_xlabel("ID of the biological sample")
ax[0].set_title("Distribution of biological samples")
for tick in ax[0].get_xticklabels():
    tick.set_rotation(45)

ax[1] = sns.countplot(
    x="timepoint",
    hue="timepoint",
    data=sampled_data,
    ax=ax[1],
    order=tp_order,
    hue_order=tp_order,
    dodge=False,
    palette=color_palette,
)
ax[1].set_xlabel("Timepoint with respect to the treatment")
ax[1].set_title("Distribution of the different timepoints")
ax[1].legend(loc="lower right")

plt.show()
plt.close()

----

#### Sample and feature selection

We now filter out constant features and nuclei with missing features.

In [ ]:
data = preprocess_data(sampled_data, remove_constant_features=True)

---

#### Data preparation

After sampling the data, we will now prepare the data for the consecutive analysis, i.e. extracting only chrometric features and corresponding metadata information.

In [ ]:
all_chrometric_data = get_chrometric_data(
    data,
    proteins=["gh2ax", "lamin", "cd3"],
    exclude_dna_int=True,
)

sample_labels = data.loc[:, "sample"]
tp_labels = data.loc[:, "timepoint"]

Finally, we remove highly correlated features (Pearson $\rho > 0.8$) from the chrometric features.

In [ ]:
chrometric_data = remove_correlated_features(all_chrometric_data, threshold=0.8)

---

## 3. Panels

Now we generate the individual panels for figure 3 of the paper.


### 3a. Visualization of changes of nuclear phenotypes in different cancer types

First, we provide a visual representation of the different nuclear phenotypes in health and cancer. To this end, we will randomly sample 36 nuclei from each of the three cancer types and plot a corresponding montage of the max-z projected DNA images. To visualize size differences each nuclei is padded to a size of 150x150 pixels. Note that the nuclei images were obtained from range-normalized DAPI images. The range normalization was used to mitigate batch effects.

In [ ]:
image_file_path = "preprocessed/full_pipeline/segmentation/nuclei_images"
sampled_tp1_images = get_random_images(
    data.loc[data.timepoint == "prior"],
    image_file_path,
    data_dir_col="data_dir",
    n_images=16,
    seed=1234,
    file_ending=".tif",
    file_name_col="file_name",
)

sampled_tp2_images = get_random_images(
    data.loc[data.timepoint == "during"],
    image_file_path,
    data_dir_col="data_dir",
    n_images=16,
    seed=1234,
    file_ending=".tif",
    file_name_col="file_name",
)

sampled_tp3_images = get_random_images(
    data.loc[data.timepoint == "end"],
    image_file_path,
    data_dir_col="data_dir",
    n_images=16,
    seed=1234,
    file_ending=".tif",
    file_name_col="file_name",
)

#### Prior treatment population

In [ ]:
fig_tp1, ax_tp1 = plot_montage(
    sampled_tp1_images,
    pad_size=150,
    mask_nuclei=True,
    cmap="inferno",
    nrows=4,
    ncols=4,
)
fig_tp1.set_facecolor(color_palette["prior"])

#### During treatment population

In [ ]:
fig_tp2, ax_tp2 = plot_montage(
    sampled_tp2_images,
    pad_size=150,
    mask_nuclei=True,
    cmap="inferno",
    ncols=4,
    nrows=4,
)
fig_tp2.set_facecolor(color_palette["during"])

#### End of treatment population

In [ ]:
fig_tp3, ax_tp3 = plot_montage(
    sampled_tp3_images,
    pad_size=150,
    mask_nuclei=True,
    cmap="inferno",
    ncols=4,
    nrows=4,
)
fig_tp3.set_facecolor(color_palette["end"])

---

### 3b. Parametric analysis captures captures differences of PBMCs at different timepoints of the proton therapy treatment

The montage already indicate significant changes in particular between the chrometric pheontype of the PBMCs prior the treatment and the end of it. We will now turn to the assessment of the parametric descriptions of the nuclear phenotypes of the PBMCs at those different timepoints. To this end, we first visualize the data set using a tSNE plot to assess potential large-scale differences between the timepoint populations and individual patient samples.

In [ ]:
chrometric_embs = get_tsne_embs(chrometric_data)
chrometric_embs["timepoint"] = np.array(tp_labels)
chrometric_embs["sample"] = np.array(sample_labels)

In [ ]:
fig, ax = plt.subplots(figsize=[9, 6])
ax = sns.scatterplot(
    data=chrometric_embs,
    x="tSNE 1",
    y="tSNE 2",
    hue="timepoint",
    hue_order=tp_order,
    ax=ax,
    s=16,
    marker="o",
    palette=color_palette,
)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=[9, 6])
ax = sns.scatterplot(
    data=chrometric_embs,
    x="tSNE 1",
    y="tSNE 2",
    hue="sample",
    style="timepoint",
    style_order=tp_order,
    hue_order=sample_order,
    ax=ax,
    s=16,
    marker="o",
    palette="tab20",
)
plt.legend(
    bbox_to_anchor=(1.02, 0.5), loc="center left", borderaxespad=0, title="sample"
)
plt.show()

The tSNE plot also shows that especially the chrometric phenotypes of the PBMCs at the intermediate timepoint of the proton therapy look different from the prior and the end of treatment population.
The patient samples are fairly well mixed with probably the small exception of the PBMCs of P29 and P22.

---

#### Classification of the different cancer types.

To quantify the separability of the timepoint populations using the chrometric phenotypes of the PBMCs of the different cancer patients we perform a 10-fold stratified cross-validation analysis using a RandomForest classifier. The classifier provides a simple non-linear classification model which also yields an importance measure for the individual chrometric features indicating which ones are most different between the three populations.

##### Nuclei split

At first we will split the data randomly on a nuclei-basis, i.e. nuclei of the same patient will be likely included in both the training and the test sets.

In [ ]:
rfc = RandomForestClassifier(
    n_estimators=500, n_jobs=10, random_state=seed, class_weight="balanced"
)

In [ ]:
tp_cv_conf_mtx_nuclei = get_cv_conf_mtx(
    estimator=rfc,
    features=chrometric_data,
    labels=tp_labels,
    scale_features=False,
    n_folds=10,
    order=tp_order,
)
normalized_cv_conf_mtx_nuclei = tp_cv_conf_mtx_nuclei.divide(
    tp_cv_conf_mtx_nuclei.sum(axis=1), axis=0
)

In [ ]:
fig, ax = plt.subplots(figsize=[5, 4])
ax = sns.heatmap(
    normalized_cv_conf_mtx_nuclei,
    annot=True,
    fmt=".4f",
    cmap="viridis",
    vmin=0,
    vmax=1,
    # cbar=False,
)
ax.set_xlabel("Predicted treatment timepoint")
ax.set_ylabel("True treatment timepoint")
plt.show()

The classifier is able to accurately distinguish nuclei from the timepoint populations. The confusion matrix also shows that the classifier does particularly better on distinguishing PBMCs from the during the treatment from the other two timepoints, where those are more often confused for one another. Nonetheless, all timepoints seem to feature fairly different chrometric cell state distribution of the PBMCs.

To further assess the similarity of the different timepoint distribution and their linear separability we use a linear discriminant analysis.

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=2)
lda_tp_cv_conf_mtx_nuclei = get_cv_conf_mtx(
    estimator=lda,
    features=chrometric_data,
    labels=tp_labels,
    scale_features=True,
    n_folds=10,
    order=tp_order,
)
lda_normalized_cv_conf_mtx_nuclei = lda_tp_cv_conf_mtx_nuclei.divide(
    lda_tp_cv_conf_mtx_nuclei.sum(axis=1), axis=0
)

In [ ]:
fig, ax = plt.subplots(figsize=[5, 4])
ax = sns.heatmap(
    lda_normalized_cv_conf_mtx_nuclei,
    annot=True,
    fmt=".4f",
    cmap="viridis",
    vmin=0,
    vmax=1,
    # cbar=False,
)
ax.set_xlabel("Predicted treatment timepoint")
ax.set_ylabel("True treatment timepoint")
plt.show()

In [ ]:
lda_transformed = pd.DataFrame(
    lda.fit(chrometric_data, tp_labels).transform(chrometric_data),
    columns=["LD 1", "LD 2"],
    index=chrometric_data.index,
)
lda_transformed["timepoint"] = np.array(tp_labels)
lda_transformed["sample"] = np.array(sample_labels)
g = sns.jointplot(
    data=lda_transformed,
    x="LD 1",
    y="LD 2",
    hue="timepoint",
    s=8,
    hue_order=tp_order,
    height=6,
    palette=color_palette,
    xlim=[-5, 5],
    ylim=[-4, 4],
)

In [ ]:
np.sum(np.abs(lda_transformed.loc[:, "LD 1"]) > 5) + np.sum(
    np.abs(lda_transformed.loc[:, "LD 2"]) > 4
)

The above LDA plot suggests in addition to the confusion matrix that the indeed the population of the PBMCs prior and the end of the treatment of the proton therapy look more alike compared to those during the treatment.

Note that in the above plot 6 outlier sample is excluded for better visualization of the population differences.

---
#### Patient split


While the previous analysis assess the level of differences of the chrometric phenotypes of the PBMCs between the different treatment timepoints, the classifier can make use of patient specific characteristics during the classification. In a diagnostic use case such information would not be available. To evaluate how well a classifier would be able to predict for unseen patient the corresponding treatment timepoint simply based on the chrometric phenotypes of the PBMCs, we also assess the class separability using a stratified 5-fold patient-cross-validation approach. Thereby at each iteration 1-2 of the patients of each timepoint are hold out for the test set.

In [ ]:
tp_cv_conf_mtx_patient = get_cv_conf_mtx(
    estimator=rfc,
    features=chrometric_data,
    labels=tp_labels,
    groups=sample_labels,
    scale_features=False,
    n_folds=10,
    order=tp_order,
)
normalized_cv_conf_mtx_patient = tp_cv_conf_mtx_patient.divide(
    tp_cv_conf_mtx_patient.sum(axis=1), axis=0
)

In [ ]:
fig, ax = plt.subplots(figsize=[5, 4])
ax = sns.heatmap(
    normalized_cv_conf_mtx_patient,
    annot=True,
    fmt=".4f",
    cmap="viridis",
    vmin=0,
    vmax=1,
    annot_kws={"size": 16, "weight": "bold"},
    # cbar=False,
)
ax.set_xlabel("Predicted treatment timepoint")
ax.set_ylabel("True treatment timepoint")
plt.show()

In [ ]:
groupkfold = StratifiedGroupKFold(n_splits=10)
cv_bacs = cross_val_score(
    rfc,
    cv=groupkfold,
    X=chrometric_data,
    y=tp_labels,
    groups=sample_labels,
    scoring="balanced_accuracy",
    n_jobs=10,
)
print("Balanced accuracy: {} (+/- {})".format(np.mean(cv_bacs), np.std(cv_bacs)))

In [ ]:
test_samples = []
for train_idx, test_idx in groupkfold.split(chrometric_data, tp_labels, sample_labels):
    test_samples.append("_".join(np.unique(sample_labels[test_idx])))
cv_bac_df = pd.DataFrame(cv_bacs, columns=["BAC"])
cv_bac_df["Test samples"] = test_samples
fig, ax = plt.subplots(figsize=[6, 4])
ax = sns.barplot(x="Test samples", y="BAC", data=cv_bac_df, palette=["gray"], ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.title("Test set performance by fold")
plt.yticks(np.arange(0, 1, 0.1))
plt.show()

In [ ]:
cv_bacs = cross_val_score(
    lda,
    cv=groupkfold,
    X=chrometric_data,
    y=tp_labels,
    groups=sample_labels,
    scoring="balanced_accuracy",
    n_jobs=10,
)
print("Balanced accuracy: {} (+/- {})".format(np.mean(cv_bacs), np.std(cv_bacs)))

In general the confusion matrix again shows a similar picture as seen for the nuclei-based split showing that end of treatment population show phenotypes of the PBMCs significantly different from the other two treatment timepoints.

---

### 3c. Nuclear chromatin biomarkers identifying cancer populations

#### Feature importance

We have validated that there are significant differences between the individual treatment timepoints in particular when comparing PBMCs at the end of the treatment with those of during respectively prior the treatment. We next assess the implicit feature importance of a RandomForest classifier trained on the task to distinguish between the timepoint populations types in order to get an idea of the features are most indicative for the treatment effect.

In [ ]:
fig, ax = plot_feature_importance_for_estimator(
    rfc,
    chrometric_data,
    tp_labels,
    scale_features=False,
    feature_color_dict=feature_color_dict,
    n_features=15,
    cmap=["gray"],
    figsize=[2, 1],
)

The analysis suggests that the features that changes the most between the individual treatment populations is the size of the nucleus as well as the heterochromatin content as well as the shape of the overall DNA intensity distribution inside the nucleus in 2D.

The previously shown feature importance plots already suggest a number of candidate chrometric biomarkers that capture the differences of the nuclear phenotypes of the PBMCs during the treatment. We now run marker screen by testing for differential distributions of the individual chrometric features between the different treatment timepoint populations. To this end, we apply a t-test to test for difference in the means and adjust for multiple testing using the Benjamini-Hochberg procedure.

In [ ]:
marker_screen_results = find_markers(chrometric_data, tp_labels)

#### Prior treatment

At first we look at the features whose mean is significantly different prior the proton therapry treatment compared to during respectively the end of it.

In [ ]:
marker_screen_results.loc[marker_screen_results.label == "prior"].head(10)

We find that the PBMCs prior the treatment are average slightly larger, the projected DNA image and a more variable curvature of the nuclear boundary of the projected nuclear mask.

---

#### During treatment

Next we look at the features whose mean is significantly different in the during treatment population.

In [ ]:
marker_screen_results.loc[marker_screen_results.label == "during"].head(10)

The during treamtent population seems to feature PBMCs that are slightly larger and have a significantly larger heterochromatin content. Additionally, the skewness and kurtosis of the DNA distribution of the projected DNA image is reduced.

---

#### End of treatment

Finally, we also evaluate the chrometric phenotype of PBMCs at the end of the proton therapy treatment.

In [ ]:
marker_screen_results.loc[marker_screen_results.label == "end"].head(10)

The PBMCs at the end of the treatment have on average smaller nuclei with less heterochromatin content and whose DNA distribution shows a significantly increased skewness and kurtosis.

---

As a joint proxy to study the alterations in size, we focus at the nuclear volume, the variation in the shape by the concavity of the nucleus and the change in chromatin compaction by the relative heterochromatin to euchromatin ratio, additionally we observe differences in the curvature. Finally, the shape of the DNA intensity distributions of the z-projected nucleus are significantly different. To visualize those differences, we look at the distributions of those markers in the different cancer types.

In [ ]:
markers = [
    "volume",
    "hetero_to_euchromatin_volume_ratio",
    "std_curvature",
    "glcm_correlation_5px",
]
marker_labels = [
    r"Nuclear volume in px$^3$",
    "relative HC/EC ratio",
    "Standard deviation of the curvature",
    "Correlation of the GLCM \n with a shift of 5px (2D)",
]
plot_timepoint_markers_dist(
    data, markers, marker_labels, cut=0, palette=color_palette, figsize=[4, 4]
)

In [ ]:
plot_timepoint_markers_dist(
    data, markers, marker_labels, cut=0, plot_type="bar", palette=color_palette
)

---

### 3d. Proteomic differences of PBMCs in cancer

Finally, we also assess the proteomic differences between the different treatment timepoint populations. To this end, we plot the relative Lamin and gH2AX expression measured by the sum of the intensities of the corresponding imaging channels normalized by the nuclear volume. Additionally, we plot the number of identified gH2AX foci which are computed as the local maxima peaks found in the corresponding channel images.

Note that those features are only available for the first data set that was stained for those proteins.

In [ ]:
markers = [
    "rel_lamin_3d_int",
    "rel_gh2ax_3d_int",
    "gh2ax_foci_count",
    "gh2ax_sum_foci_area",
    "gh2ax_avg_foci_area",
]
marker_labels = [
    "Volume-normalized nuclear\nLamin A/C intensity",
    "Normalized nuclear\n" r"$\gamma$H2AX intensity",
    r"Number of $\gamma$H2AX foci",
    r"Sum of the $\gamma$H2AX foci area",
    r"Average size of the $\gamma$H2AX foci",
]
plot_timepoint_markers_dist(
    data,
    markers,
    marker_labels,
    quantiles=None,
    cut=0,
    plot_type="bar",
    palette=color_palette,
)

In [ ]:
markers = [
    "rel_lamin_3d_int",
    "rel_gh2ax_3d_int",
    "gh2ax_foci_count",
    "gh2ax_sum_foci_area",
    "gh2ax_avg_foci_area",
]
marker_labels = [
    "Volume-normalized nuclear\nLamin A/C intensity",
    "Normalized nuclear\n" r"$\gamma$H2AX intensity",
    r"Number of $\gamma$H2AX foci",
    r"Sum of the $\gamma$H2AX foci area",
    r"Average size of the $\gamma$H2AX foci",
]
plot_timepoint_markers_dist(
    data,
    markers,
    marker_labels,
    quantiles=None,
    cut=0,
    plot_type="violin",
    palette=color_palette,
)

---

## 4. Supplemental

In [ ]:
us_chrometric_data = get_chrometric_data(
    all_data,
    proteins=["gh2ax", "lamin", "cd3"],
    exclude_dna_int=True,
)

us_sample_labels = all_data.loc[:, "sample"]
us_tp_labels = all_data.loc[:, "timepoint"]

us_chrometric_data = remove_correlated_features(us_chrometric_data, threshold=0.8)

us_chrometric_embs = get_tsne_embs(us_chrometric_data)
us_chrometric_embs["timepoint"] = np.array(us_tp_labels)
us_chrometric_embs["sample"] = np.array(us_sample_labels)

In [ ]:
fig, ax = plt.subplots(figsize=[9, 6])
ax = sns.scatterplot(
    data=us_chrometric_embs,
    x="tSNE 1",
    y="tSNE 2",
    hue="timepoint",
    hue_order=tp_order,
    ax=ax,
    s=8,
    marker="o",
    palette=color_palette,
)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=[9, 6])
ax = sns.scatterplot(
    data=us_chrometric_embs,
    x="tSNE 1",
    y="tSNE 2",
    hue="sample",
    style="timepoint",
    style_order=tp_order,
    hue_order=sample_order,
    ax=ax,
    s=8,
    marker="o",
    palette="tab20",
)
plt.legend(
    bbox_to_anchor=(1.02, 0.5), loc="center left", borderaxespad=0, title="sample"
)
plt.show()